In [ ]:
# import packages

import pandas as pd
import boto3
import numpy as np
from sklearn import preprocessing


In [ ]:
# Read the CSV with pandas

lns = pd.read_csv('s3://xxxxxxx/lns.csv')

In [ ]:
# Assign unique id to each row

lns['unique_id'] = np.arange(0,len(lns))

lns.to_csv('s3://xxxxxxxxxxx/lns.csv')

lns

In [ ]:
# Define Label Encoder - label encoder is used to make pseudonyms.
le = preprocessing.LabelEncoder()

# Create Pseudonym column for ANum
lns['ANUM_PSEUD'] = le.fit_transform(lns['ANUMBER'])
lns['ANUM_PSEUD'] = lns['ANUM_PSEUD'].astype(str) + '_A_NUM'

# Create pseud col for Ref num

lns['REFNUM_PSEUD'] = le.fit_transform(lns['REFNUM'])
lns['REFNUM'] = lns['REFNUM_PSEUD'].astype(str) + '_REFNUM'

# Create Pseud col for CSL Name

lns['CSL_NAME_PSEUD'] = le.fit_transform(lns['CSL_NAME'])
lns['CSL_NAME_PSEUD'] = lns['CSL_NAME_PSEUD'].astype(str) + '_CSL_NAME'

# Create pseud col for C_Name

lns['C_NAME_PSEUD'] = le.fit_transform(lns['C_NAME'])
lns['C_NAME_PSEUD'] = lns['C_NAME_PSEUD'].astype(str) + '_C_NAME'

# Create Pseud for I_A_Num

lns['I_ANUM_PSEUD'] = le.fit_transform(lns['I_A_NUMBER'])
lns['I_ANUM_PSEUD'] = lns['I_ANUM_PSEUD'].astype(str) + 'I_ANUM'


In [ ]:
# Export 2 CSVs - one with the sensitive info, and one without

# csv 1 - sensitive

senslns = lns.to_csv(index = False)

s3 = boto3.resource('s3')
s3.Object('alpha-app-laa-fas-qc-dsa', 'SENSITIVE_lns.csv').put(Body=senslns)

In [ ]:
# CSV 2 - pseudonymised - sensitive columns dropped

pseudlns = lns.drop(columns=['ANUMBER', 'ONAME', 'LANUMBER','FNAME', 'REFNUM',
                     'I_A_NUMBER', 'IO_NAME', 'CODE', 'C_USER_NAME','C_NAME'])

# Convert dataframe to csv

pseudlns = pseudlns.to_csv(index=False)

# Save CSV to Amazon S3 Cloud

s3.Object('xxxxxxxxxxxxxxxxxxx', 'Pseud_lns.csv').put(Body=pseudlns)